In [1]:
import tensorflow as tf
import string
import requests
import csv
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
sentences = []
with open('judgesSentencesOnly.csv', newline='') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        sentences.append(row)
sents = [sent[0] for sent in sentences if len(sent[0]) > 2]
print(sents[:10])

['\ufeffSentence', ' 1 September 23, 1932, the Secretary of Agriculture, acting under the Packers and Stockyards Act, 1921,1 ordered an inquiry and gave notice of a hearing to determine the reasonableness of rates charged by market agencies doing business at the Union Stockyards in Chicago', ' After protracted hearings and argument, he made findings of fact, announced his conclusion that the existing rates were unreasonable, and fixed new maximum rates', ' The appellants, who conduct market agencies, petitioned for rehearing', ' This the secretary denied, but by a supplemental order he increased some rates', ' An amended petition for rehearing was dismissed and the appellants then filed their bill in the District Court seeking an injunction against enforcement of the original and supplemental orders', ' The case was heard by three judges, who granted an interlocutory injunction', " At final hearing, the appellants offered in evidence the record of the proceedings before the secretary a

In [3]:
sents = " ".join(sents)
sents[:30]

'\ufeffSentence  1 September 23, 193'

In [4]:
def clean_text(data):
  words = data.split()
  table = str.maketrans('', '', string.punctuation)
  words = [word.translate(table) for word in words]
  words = [word for word in words if word.isalpha() ]
  words = [word.lower() for word in words]
  return words

cleanSents = clean_text(sents)
cleanSents[:10]

['september',
 'the',
 'secretary',
 'of',
 'agriculture',
 'acting',
 'under',
 'the',
 'packers',
 'and']

In [5]:
predLength = 15
groups = []
for i in range(predLength, len(cleanSents)):
    part = cleanSents[i - predLength: i]
    grouping = ' '.join(part)
    groups.append(grouping)
print(len(groups))
groups[0]

81522


'september the secretary of agriculture acting under the packers and stockyards act ordered an inquiry'

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(groups)
sequences = tokenizer.texts_to_sequences(groups)
print(type(sequences))
sequences = np.array(sequences)
print(type(sequences))

<class 'list'>
<class 'numpy.ndarray'>


In [7]:
print(np.shape(sequences))
X = sequences[:, :-1]
y = sequences[:, -1]
print(X[0])
print(y[0])

(81522, 15)
[ 663    1  123    2 1186 2739   48    1 5652    4 2738   22  579   24]
1657


In [8]:
uniqueWords = 1 + len(tokenizer.word_index)
uniqueWords

5653

In [9]:
y = to_categorical(y, num_classes=uniqueWords)

In [10]:
rnnModel = tf.keras.models.Sequential()
rnnModel.add(tf.keras.layers.Embedding(uniqueWords, 14, input_length=14))
rnnModel.add(tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.2))
rnnModel.add(tf.keras.layers.LSTM(256, return_sequences=False, dropout=0.3, recurrent_dropout=0.2))
rnnModel.add(tf.keras.layers.Dense(units = 256, activation='relu'))
rnnModel.add(tf.keras.layers.Dense(units = 256, activation='relu'))
rnnModel.add(tf.keras.layers.Dense(units = uniqueWords, activation='softmax'))
rnnModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 14, 14)            79142     
_________________________________________________________________
lstm (LSTM)                  (None, 14, 256)           277504    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 5653)              1452821   
Total params: 2,466,363
Trainable params: 2,466,363
Non-trainable params: 0
______________________________________________

In [11]:
print(X.shape)
print(y.shape)

(81522, 14)
(81522, 5653)


In [12]:
rnnModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
rnnModel.fit(X, y, batch_size=64, epochs=50)

Epoch 1/50
1274/1274 [==============================] - 150s 118ms/step - loss: 6.2524 - accuracy: 0.1178
Epoch 2/50
1274/1274 [==============================] - 158s 124ms/step - loss: 5.7961 - accuracy: 0.1449
Epoch 3/50
1274/1274 [==============================] - 157s 123ms/step - loss: 5.5798 - accuracy: 0.1534
Epoch 4/50
1274/1274 [==============================] - 165s 130ms/step - loss: 5.4417 - accuracy: 0.1628
Epoch 5/50
1274/1274 [==============================] - 163s 128ms/step - loss: 5.3124 - accuracy: 0.1726
Epoch 6/50
1274/1274 [==============================] - 152s 119ms/step - loss: 5.1948 - accuracy: 0.1811
Epoch 7/50
1274/1274 [==============================] - 160s 125ms/step - loss: 5.0943 - accuracy: 0.1886
Epoch 8/50
1274/1274 [==============================] - 161s 126ms/step - loss: 4.9993 - accuracy: 0.1943
Epoch 9/50
1274/1274 [==============================] - 162s 127ms/step - loss: 4.9097 - accuracy: 0.1970
Epoch 10/50
1274/1274 [=======================

In [13]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, numWords):
  predText = []
  for _ in range(numWords):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen=text_seq_length, truncating='pre')
    y_pred = model.predict_classes(encoded)
    predWord = ''
    for word, index in tokenizer.word_index.items():
      if index == y_pred:
        predWord = word
        break
    seed_text = seed_text + ' ' + predWord
    predText.append(predWord)
  return ' '.join(predText)

In [15]:
seed_text = groups[200]
#generate_text_seq(rnnModel, tokenizer, 14, seed_text, 15)